In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # ETL Process for Customer 360 Profile
# MAGIC This notebook performs an ETL process to create a comprehensive customer profile by integrating data from various sources.

# COMMAND ----------

import logging
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# COMMAND ----------

try:
    # Load data from Unity Catalog tables
    logger.info("Loading data from Unity Catalog tables...")
    policy_df = spark.table("catalog.source_db.policy")
    claims_df = spark.table("catalog.source_db.claims")
    demographics_df = spark.table("catalog.source_db.demographics")
    scores_df = spark.table("catalog.source_db.scores")
    aiml_insights_df = spark.table("catalog.source_db.aiml_insights")

# COMMAND ----------

    # Data Selection and Filtering
    logger.info("Selecting relevant fields from demographics data...")
    selected_demographics_df = demographics_df.select(
        "Customer_ID", "Customer_Name", "Email", "Phone_Number", "Address", 
        "City", "State", "Postal_Code", "Date_of_Birth", "Gender", 
        "Marital_Status", "Occupation", "Income_Level", "Customer_Segment"
    )

# COMMAND ----------

    # Data Integration
    logger.info("Joining datasets...")
    joined_df = selected_demographics_df.join(
        policy_df, selected_demographics_df.Customer_ID == policy_df.customer_id, "inner"
    ).join(
        claims_df, policy_df.policy_id == claims_df.Policy_ID, "inner"
    )

# COMMAND ----------

    # Data Aggregation
    logger.info("Aggregating data...")
    aggregated_df = joined_df.groupBy("Customer_ID").agg(
        F.count("Claim_ID").alias("Total_Claims"),
        F.count("policy_id").alias("Policy_Count"),
        F.max("Claim_Date").alias("Recent_Claim_Date"),
        F.avg("Claim_Amount").alias("Average_Claim_Amount")
    )

# COMMAND ----------

    # Custom Calculations
    logger.info("Performing custom calculations...")
    final_df = aggregated_df.withColumn(
        "Age", F.floor(F.datediff(F.current_date(), "Date_of_Birth") / 365)
    ).withColumn(
        "Claim_To_Premium_Ratio", 
        F.when(F.col("total_premium_paid") != 0, F.col("Claim_Amount") / F.col("total_premium_paid")).otherwise(0)
    ).withColumn(
        "Claims_Per_Policy", 
        F.when(F.col("Policy_Count") != 0, F.col("Total_Claims") / F.col("Policy_Count")).otherwise(0)
    ).withColumn(
        "Retention_Rate", F.lit(0.85)
    ).withColumn(
        "Cross_Sell_Opportunities", F.lit("Multi-Policy Discount, Home Coverage Add-on")
    ).withColumn(
        "Upsell_Potential", F.lit("Premium Vehicle Coverage")
    )

# COMMAND ----------

    # Comprehensive Data Joining
    logger.info("Joining with scores and AI/ML insights...")
    complete_profile_df = final_df.join(
        scores_df, "Customer_ID", "inner"
    ).join(
        aiml_insights_df, "Customer_ID", "inner"
    )

# COMMAND ----------

    # Output Configuration
    logger.info("Writing the final dataset to Unity Catalog table...")
    complete_profile_df.write.format("delta").mode("overwrite").saveAsTable("catalog.target_db.customer_360_profile")

    logger.info("ETL process completed successfully.")

except Exception as e:
    logger.error(f"An error occurred during the ETL process: {e}")
    raise
